
<div style="background-color: #d5f5e3; padding: 15px; border-radius: 10px; font-family: Garamond; color: #4a235a; font-size: 20px;">

<h2 style="text-align: center; font-weight: normal;">Dependencies and their versions</h2>
</ul>

</div>


In [3]:
import torch
import wandb
import os
import numpy as np
import torch.nn as nn
import pytorch_lightning as pl
import os, gc, torch
from torch.utils.data import DataLoader, random_split, SubsetRandomSampler
from torchvision import datasets, transforms, models
from pathlib import Path
from typing import Optional, Tuple
import torch.optim as optim
import torch.nn.functional as F
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from torchsummary import summary
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.tuner.tuning import Tuner
from torchvision.models.vision_transformer import vit_b_16
from typing import Literal

wandb.login(key = '5df7feeffbc5b918c8947f5fe4bab4b67ebfbb69')
# key = 5df7feeffbc5b918c8947f5fe4bab4b67ebfbb69

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/indramandal/.netrc
wandb: Currently logged in as: ed24s014 (ed24s014-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

<div style="background-color: #d5f5e3; padding: 15px; border-radius: 10px; font-family: Garamond; color: #4a235a; font-size: 20px;">

<h2 style="text-align: center; font-weight: normal;">Part - B / Question 1 (5 Marks)</h2>

<p>In most deep learning applications, instead of training a model from scratch, you would use a model <b>pre-trained on a similar or related task/dataset</b>.</p>

<p>From <b>torchvision</b>, you can load <b>any one</b> of the following pre-trained models:</p>

<ul>
<li>GoogLeNet</li>
<li>InceptionV3</li>
<li>ResNet50</li>
<li>VGG</li>
<li>EfficientNetV2</li>
<li>VisionTransformer</li>
<li>... or any other model available</li>
</ul>

<p>These models are pre-trained on the <b>ImageNet</b> dataset, which includes many animal images. Hence, it stands to reason that using a model pre-trained on ImageNet could be helpful for your task.</p>

<p>You are required to:</p>

<ul>
<li><b>Load a pre-trained model</b> from torchvision, and</li>
<li><b>Fine-tune it</b> using the <b>iNaturalist dataset</b> you used in the previous question.</li>
</ul>

<p>Simply put, instead of <b>randomly initializing the weights</b> of the network, you will <b>use the weights obtained from training on ImageNet</b> (torchvision directly provides these weights).</p>

</div>


### -> Custom Module for Dataloading and Transformations

In [ ]:

"""
This is a custom data module built using PyTorch Lightning, designed to handle dataset loading, transformation, and splitting. 
The module is flexible and allows for the use of data augmentation, custom image sizes, and a validation split during training. 
It simplifies dataset handling for image classification tasks and integrates seamlessly with PyTorch Lightning's training loops.

### Class Overview:
The `CustomDataModule` class inherits from `pl.LightningDataModule`, which is a PyTorch Lightning class designed to organize dataset loading and
 transformation. It contains methods for loading training and validation datasets, defining transformations, and generating data loaders.

### Arguments:
- `data_dir` (str): Path to the directory containing the dataset (should have subdirectories for each class).
- `image_size` (Tuple[int, int]): The target image size (height, width) to which all images will be resized.
- `batch_size` (int): The number of samples per batch for the data loader.
- `val_split` (float): The fraction of the dataset to use for validation (defaults to 20% of the data).
- `use_augmentation` (bool): A flag that indicates whether to apply data augmentation for training images.
- `num_workers` (int): The number of worker processes for loading data.
- `seed` (int): The random seed for reproducibility.

### Main Methods:
- `__init__`: Initializes the data module with provided parameters and prepares transformations for both training and testing.
- `_get_train_transform`: Defines the transformation pipeline for the training dataset, including data augmentation (optional).
- `_get_test_transform`: Defines the transformation pipeline for the validation and test datasets (no augmentation).
- `setup`: Loads and splits the dataset into training and validation sets, applying appropriate transformations.
- `train_dataloader`: Returns the data loader for training data.
- `val_dataloader`: Returns the data loader for validation data.
- `test_dataloader`: Returns the data loader for test data.

This class is primarily used to handle dataset-specific preprocessing, data augmentation, and splitting into training and validation sets for 
deep learning models using PyTorch Lightning.
"""

class CustomDataModule(pl.LightningDataModule):
    def __init__(
        self,
        data_dir: str,
        image_size: Tuple[int, int] = (224, 224),
        batch_size: int = 64,
        val_split: float = 0.2,
        use_augmentation: bool = False,
        num_workers: int = 2,
        seed: int = 42
    ):
        """
        Custom Data Module for handling dataset loading, transformation, and splitting.

        Args:
            data_dir (str): Path to dataset directory.
            image_size (Tuple[int, int]): Target image size (height, width).
            batch_size (int): Batch size for DataLoader.
            val_split (float): Fraction of training data to use for validation.
            use_augmentation (bool): Whether to apply data augmentation.
            num_workers (int): Number of workers for DataLoader.
            seed (int): Random seed for reproducibility.
        """
        super().__init__()
        self.data_dir = Path(data_dir)
        self.image_size = image_size
        self.batch_size = batch_size
        self.val_split = val_split
        self.use_augmentation = use_augmentation
        self.num_workers = num_workers
        self.seed = seed
        self.class_names = []

        # Define transforms
        self.train_transform = self._get_train_transform()
        self.test_transform = self._get_test_transform()

    def _get_train_transform(self):
        if self.use_augmentation:
            return transforms.Compose([
                transforms.RandomResizedCrop(self.image_size),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
                transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), shear=10),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        else:
            return transforms.Compose([
                transforms.Resize(self.image_size),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])


    def _get_test_transform(self):
        """Defines transformation pipeline for validation and test data."""
        return transforms.Compose([
            transforms.Resize(self.image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def setup(self, stage: Optional[str] = None):
        """Loads datasets and applies transformations."""
        # Set manual seeds for reproducibility
        torch.manual_seed(self.seed)
        np.random.seed(self.seed)

        # Load full dataset with train transform
        full_dataset = datasets.ImageFolder(root=self.data_dir, transform=self.train_transform)
        self.class_names = full_dataset.classes

        # Compute split sizes
        total_size = len(full_dataset)
        val_size = int(total_size * self.val_split)
        train_size = total_size - val_size

        # Split dataset into training and validation sets
        train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

        # Apply test transform to validation dataset
        val_dataset.dataset.transform = self.test_transform

        # Assign datasets
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset

    def train_dataloader(self):
        """Returns DataLoader for training data."""
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers, persistent_workers=True)

    def val_dataloader(self):
        """Returns DataLoader for validation data."""
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers, persistent_workers=True)

    def test_dataloader(self, test_dir: Optional[str] = None):
        """Returns DataLoader for test data."""
        test_path = Path(test_dir) if test_dir else self.data_dir.parent / "val"
        test_dataset = datasets.ImageFolder(root=test_path, transform=self.test_transform)
        return DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)


### -> Pretrained Models Loading and applying all Freezing Strategies

In [ ]:
"""
A PyTorch Lightning module designed for flexible image classification using a variety of popular backbone architectures.
This class encapsulates:

1. **Initialization & Hyperparameters**  
   - Stores learning rate and training strategy settings.  
   - Dynamically replaces the model’s final classifier layer to match the target number of classes.

2. **Model Loading**  
   - Supports ResNet‑50, VGG‑16, Inception‑V3, GoogleNet, EfficientNet‑V2‑S, and ViT‑B‑16.  
   - Fetches pretrained weights and swaps out the head for a new linear layer sized to `num_classes`.

3. **Fine‑tuning Strategies**  
   - **freeze_all**: Locks every pretrained weight except the newly added classifier.  
   - **freeze_partial**: Freezes all but the last `k_layers`, then unfreezes the final classification head.  
   - **unfreeze_all**: Allows gradient updates on every parameter.

4. **Training & Validation Steps**  
   - Implements `training_step` and `validation_step` to compute cross‑entropy loss and classification accuracy.  
   - Logs metrics (`train_loss`, `train_acc`, `val_loss`, `val_acc`) to the progress bar.

5. **Optimizer Configuration**  
   - Uses Adam optimizer on only those parameters marked for training, with the stored learning rate.

This modular design makes it straightforward to switch backbones, adjust fine‑tuning depth, and monitor performance.
"""

class LitClassifier(pl.LightningModule):
    def __init__(
        self,
        model_name: str,
        num_classes: int,
        lr: float = 1e-3,
        finetune_strategy: Literal['freeze_all', 'freeze_partial', 'unfreeze_all'] = 'freeze_all',
        k_layers: int = 0
    ):
        super().__init__()
        self.save_hyperparameters()
        self.lr = lr

        # Load the selected pretrained model and adapt its classifier
        self.model = self._load_model(model_name, num_classes)

        # Apply the chosen fine-tuning policy
        self._apply_finetune_strategy(finetune_strategy, k_layers)

    def _load_model(self, model_name: str, num_classes: int) -> nn.Module:
        if model_name == 'resnet50':
            model = models.resnet50(pretrained=True)
            model.fc = nn.Linear(model.fc.in_features, num_classes)

        elif model_name == 'vgg16':
            model = models.vgg16(pretrained=True)
            model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

        elif model_name == 'inception_v3':
            model = models.inception_v3(pretrained=True, aux_logits=True)
            model.fc = nn.Linear(model.fc.in_features, num_classes)
            model.aux_logits = False  # disable auxiliary output during inference

        elif model_name == 'googlenet':
            model = models.googlenet(pretrained=True, aux_logits=True)
            model.fc = nn.Linear(model.fc.in_features, num_classes)
            model.aux_logits = False

        elif model_name == 'efficientnet_v2_s':
            model = models.efficientnet_v2_s(pretrained=True)
            model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

        elif model_name == 'vit_b_16':
            model = vit_b_16(pretrained=True)
            model.heads.head = nn.Linear(model.heads.head.in_features, num_classes)

        else:
            raise ValueError(f"Unsupported model: {model_name}")

        return model

    def _apply_finetune_strategy(self, strategy: str, k_layers: int):
        layers = list(self.model.children())

        if strategy == 'freeze_all':
            for p in self.model.parameters():
                p.requires_grad = False
            self._unfreeze_final_classifier()

        elif strategy == 'freeze_partial':
            # Freeze early layers, leave last `k_layers` unfrozen
            cutoff = len(layers) - k_layers
            for idx, layer in enumerate(layers):
                if idx < cutoff:
                    for p in layer.parameters():
                        p.requires_grad = False
            self._unfreeze_final_classifier()

        elif strategy == 'unfreeze_all':
            for p in self.model.parameters():
                p.requires_grad = True

    def _unfreeze_final_classifier(self):
        # Ensure only the final classification head is trainable
        if hasattr(self.model, 'fc'):
            for p in self.model.fc.parameters():
                p.requires_grad = True
        elif hasattr(self.model, 'classifier'):
            for p in self.model.classifier.parameters():
                p.requires_grad = True
        elif hasattr(self.model, 'heads'):
            for p in self.model.heads.parameters():
                p.requires_grad = True

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logits = self.model(x)
        # Some architectures return (primary_output, auxiliary_output)
        if isinstance(logits, tuple):
            logits = logits[0]
        return logits

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def configure_optimizers(self):
        # Only optimize parameters that require gradients
        trainable_params = filter(lambda p: p.requires_grad, self.parameters())
        return torch.optim.Adam(trainable_params, lr=self.lr)


<div style="background-color: #d5f5e3; padding: 15px; border-radius: 10px; font-family: Garamond; color: #4a235a; font-size: 20px;">

<h2 style="text-align: center; font-weight: normal;">Question 3 (10 Marks)</h2>

<p>Now, <b>fine-tune the model</b> using <b>any one</b> of the listed strategies you discussed above.</p>

<p>After completing the fine-tuning experiments, you are required to:</p>

<ul>
<li><b>Write down insightful inferences</b> based on your observations.</li>
<li><b>Compare</b> the results of:
    <ul>
        <li><b>Training a model from scratch</b> versus</li>
        <li><b>Fine-tuning a large pre-trained model</b></li>
    </ul>
</li>
</ul>

<p>Focus on making meaningful comparisons based on performance metrics such as accuracy, training time, convergence behavior, and generalization ability.</p>

</div>


### -> Fully Customizable Pretrained Model's running codes for all different type of Pretrained models mentioned in the question with all Different type of Freezing Strategies

In [ ]:


# Step 1: Set hyperparameters here
MODEL_NAME = "efficientnet_v2_s"  # Options: resnet50, vgg16, inception_v3, googlenet, efficientnet_v2_s, vit_b_16
USE_AUGMENTATION = True
FINETUNE_STRATEGY = "freeze_partial"  # Options: freeze_all, freeze_partial, unfreeze_all
K_LAYERS = 1  # Only used when freeze_partial
LEARNING_RATE = 1e-4
BATCH_SIZE = 64
IMAGE_SIZE = {
    "resnet50": (224, 224),
    "vgg16": (224, 224),
    "inception_v3": (299, 299),
    "googlenet": (224, 224),
    "efficientnet_v2_s": (384, 384),
    "vit_b_16": (224, 224),
}[MODEL_NAME]
NUM_EPOCHS = 16
SEED = 42

# Step 2: Initialize the DataModule
data_module = CustomDataModule(
    data_dir="/Users/indramandal/Documents/VS_CODE/DA6401/DA6401_Assignment_2/inaturalist_12K/train",
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    val_split=0.2,
    use_augmentation=USE_AUGMENTATION,
    seed=SEED
)
data_module.setup(stage="fit")


os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
gc.collect()
torch.cuda.empty_cache()

# Step 3: Initialize the model
model = LitClassifier(
    model_name=MODEL_NAME,
    num_classes=len(data_module.class_names),
    lr=LEARNING_RATE,
    finetune_strategy=FINETUNE_STRATEGY,
    k_layers=K_LAYERS
)

# Step 4: Print model summary
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print("Detailed Model Summary:")
summary(model, input_size=(3, *IMAGE_SIZE))


# Step 5: Configure Trainer
trainer = Trainer(
    max_epochs=10,
    precision="16-mixed",
    accumulate_grad_batches=2,  # new
    callbacks=[EarlyStopping(monitor="val_acc", mode="max", patience=3)]
)

# Step 6: Train the model
trainer.fit(model, datamodule=data_module)


/Users/indramandal/Documents/VS_CODE/DA6401/DA6401_Assignment_2/env/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/indramandal/Documents/VS_CODE/DA6401/DA6401_Assignment_2/env/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Detailed Model Summary:


Using 16bit Automatic Mixed Precision (AMP)
/Users/indramandal/Documents/VS_CODE/DA6401/DA6401_Assignment_2/env/lib/python3.9/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 24, 192, 192]             648
       BatchNorm2d-2         [-1, 24, 192, 192]              48
              SiLU-3         [-1, 24, 192, 192]               0
            Conv2d-4         [-1, 24, 192, 192]           5,184
       BatchNorm2d-5         [-1, 24, 192, 192]              48
              SiLU-6         [-1, 24, 192, 192]               0
   StochasticDepth-7         [-1, 24, 192, 192]               0
       FusedMBConv-8         [-1, 24, 192, 192]               0
            Conv2d-9         [-1, 24, 192, 192]           5,184
      BatchNorm2d-10         [-1, 24, 192, 192]              48
             SiLU-11         [-1, 24, 192, 192]               0
  StochasticDepth-12         [-1, 24, 192, 192]               0
      FusedMBConv-13         [-1, 24, 192, 192]               0
           Conv2d-14           [-1, 96,

/Users/indramandal/Documents/VS_CODE/DA6401/DA6401_Assignment_2/env/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

  | Name  | Type         | Params | Mode 
-----------------------------------------------
0 | model | EfficientNet | 20.2 M | train
-----------------------------------------------
12.8 K    Trainable params
20.2 M    Non-trainable params
20.2 M    Total params
80.761    Total estimated model params size (MB)
714       Modules in train mode
0         Modules in eval mode


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/indramandal/Documents/VS_CODE/DA6401/DA6401_Assignment_2/env/lib/python3.9/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Epoch 9: 100%|██████████| 125/125 [04:02<00:00,  0.51it/s, v_num=73, train_loss=0.987, train_acc=0.750, val_loss=0.946, val_acc=0.777]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 125/125 [04:03<00:00,  0.51it/s, v_num=73, train_loss=0.987, train_acc=0.750, val_loss=0.946, val_acc=0.777]


### -> Wandb Sweep Configuration 

In [8]:
sweep_config = {
    "method": "bayes",  # or "grid"
    "metric": {
        "name": "val_acc",
        "goal": "maximize"
    },
    "parameters": {
        "model_name": {
            "values": ["resnet50", "vgg16", "inception_v3", "googlenet", "efficientnet_v2_s", "vit_b_16"]
        },
        "lr": {
            "values": [1e-3, 1e-4, 1e-5]
        },
        "batch_size": {
            "values": [16, 32]
        },
        "finetune_strategy": {
            "values": ["freeze_all", "freeze_partial", "unfreeze_all"]
        },
        "k_layers": {
            "values": [1, 2, 3]
        },
        "use_augmentation": {
            "values": [True, False]
        }
    }
}


### -> Sweep Train Wrapper Function for logging data for all Different Models and Hyperparameters with all differernt type of Freezing Strategies

In [9]:
def sweep_train(config=None):
    with wandb.init(config=config):
        config = wandb.config

        IMAGE_SIZE = {
            "resnet50": (224, 224),
            "vgg16": (224, 224),
            "inception_v3": (299, 299),
            "googlenet": (224, 224),
            "efficientnet_v2_s": (384, 384),
            "vit_b_16": (224, 224),
        }[config.model_name]

        #  Step 2: Initialize the DataModule
        data_module = CustomDataModule(
            data_dir="/Users/indramandal/Documents/VS_CODE/DA6401/DA6401_Assignment_2/inaturalist_12K/train",
            image_size=IMAGE_SIZE,
            batch_size=config.batch_size,
            val_split=0.2,
            use_augmentation=config.use_augmentation,
            seed=42
        )
        data_module.setup(stage="fit")


        os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
        gc.collect()
        torch.cuda.empty_cache()

        # Step 3: Initialize the model
        model = LitClassifier(
            model_name=config.model_name,
            num_classes=len(data_module.class_names),
            lr=config.lr,
            finetune_strategy=config.finetune_strategy,
            k_layers=config.k_layers
        )

        # Step 4: Print model summary
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model.to(device)
        print("Detailed Model Summary:")
        summary(model, input_size=(3, *IMAGE_SIZE))

        wandb_logger = WandbLogger(project="finetune", log_model=True)

        trainer = Trainer(
            max_epochs=15,
            precision="16-mixed",
            accumulate_grad_batches=2,
            callbacks=[EarlyStopping(monitor="val_acc", mode="max", patience=3)],
            logger=wandb_logger
        )

        trainer.fit(model, datamodule=data_module)


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="finetune")
wandb.agent(sweep_id, function=sweep_train, count=10)  # adjust count



Create sweep with ID: th8nz5u8
Sweep URL: https://wandb.ai/ed24s014-indian-institute-of-technology-madras/finetune/sweeps/th8nz5u8


wandb: Agent Starting Run: jwbl1r2m with config:
wandb: 	batch_size: 32
wandb: 	finetune_strategy: freeze_partial
wandb: 	k_layers: 2
wandb: 	lr: 1e-05
wandb: 	model_name: efficientnet_v2_s
wandb: 	use_augmentation: True


/Users/indramandal/Documents/VS_CODE/DA6401/DA6401_Assignment_2/env/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/indramandal/Documents/VS_CODE/DA6401/DA6401_Assignment_2/env/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Detailed Model Summary:


Using 16bit Automatic Mixed Precision (AMP)
/Users/indramandal/Documents/VS_CODE/DA6401/DA6401_Assignment_2/env/lib/python3.9/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


...

/Users/indramandal/Documents/VS_CODE/DA6401/DA6401_Assignment_2/env/lib/python3.9/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Epoch 1: 100%|██████████| 250/250 [02:56<00:00,  1.41it/s, v_num=1r2m, train_loss=2.250, train_acc=0.156, val_loss=2.270, val_acc=0.159] 

wandb: Ctrl + C detected. Stopping sweep.


Traceback (most recent call last):
  File "/var/folders/16/2v843_gd4x1cty4mwj7r2t780000gn/T/ipykernel_9193/2059808040.py", line 55, in sweep_train
    trainer.fit(model, datamodule=data_module)
  File "/Users/indramandal/Documents/VS_CODE/DA6401/DA6401_Assignment_2/env/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 561, in fit
    call._call_and_handle_interrupt(
  File "/Users/indramandal/Documents/VS_CODE/DA6401/DA6401_Assignment_2/env/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py", line 48, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
  File "/Users/indramandal/Documents/VS_CODE/DA6401/DA6401_Assignment_2/env/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 599, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "/Users/indramandal/Documents/VS_CODE/DA6401/DA6401_Assignment_2/env/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 1012, in _run
    results = self._r

: 

In [13]:
wandb.finish()

<div style="background-color: #d5f5e3; padding: 15px; border-radius: 10px; font-family: Garamond; color: #4a235a; font-size: 20px;">

<h2 style="text-align: center; font-weight: normal;">Useful Links</h2>

<ul>
<li><b>GitHub Repository:</b> 
<a href="https://github.com/indramandal85/DA6401_Assignment_2_CNN.git" target="_blank">Click here to view the GitHub repository</a>
</li>
<br>
<li><b>Weights & Biases (wandb) Report:</b> 
<a href="https://wandb.ai/ed24s014-indian-institute-of-technology-madras/CNN_Hyperparameter_Tuning_3/reports/Copy-of-DA6401-Assignment-2--VmlldzoxMjI2MjQyNg?accessToken=n5t71n8616wbg2go0svn1bhi8y7vznqoomthqhpy7wkiqpajhpn7e4ywxx9xodyc" target="_blank">Click here to view the WandB report</a>
</li>
</ul>

</div>
